In [2]:
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL
from obspy import read_inventory, UTCDateTime
from obspy import read_inventory
from obspy.clients.fdsn import Client

In [37]:
client = Client("IRIS")
network = 'XE'
station = 'CC03'
station_code = network+'.'+station+'..'+'BHZ'
time = "2018-04-29T00:00:00"

client = Client("IRIS")
inventory_specific = client.get_stations(network=network,station=station,channel='BHZ', level="response")
latitude = inventory_specific[0][0].latitude
longitude = inventory_specific[0][0].longitude
elevation = inventory_specific[0][0].elevation
depth = inventory_specific[0][0][0].depth
azimuth = inventory_specific[0][0][0].azimuth
dip = inventory_specific[0][0][0].dip
sample_rate = inventory_specific[0][0][0].sample_rate

# We'll first create all the various objects. These strongly follow the
# hierarchy of StationXML files.
inv = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="URSeismo-OBS")

net = Network(
    # This is the network code according to the SEED standard.
    code=network,
    # A list of stations. We'll add one later.
    stations=[],
    description="manually_remove_response_inventory",
    # Start-and end dates are optional.
    #start_date=obspy.UTCDateTime(2016, 1, 2)
)

sta = Station(
    # This is the station code according to the SEED standard.
    code=station,
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    #creation_date=obspy.UTCDateTime(2016, 1, 2),
    site=Site(name=station))

cha = Channel(
    # This is the channel code according to the SEED standard.
    code="BHZ",
    # This is the location code according to the SEED standard.
    location_code="",
    # Note that these coordinates can differ from the station coordinates.
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    depth=depth,
    azimuth=azimuth,
    dip=dip,
    sample_rate=sample_rate)

# By default this accesses the NRL online. Offline copies of the NRL can
# also be used instead
nrl = client.get_stations(network=network,station=station,channel='BHZ', level="response")
# The contents of the NRL can be explored interactively in a Python prompt,
# see API documentation of NRL submodule:
# http://docs.obspy.org/packages/obspy.clients.nrl.html
# Here we assume that the end point of data logger and sensor are already
# known:
datetime = UTCDateTime(time)
response = nrl.get_response(station_code, datetime)


# Now tie it all together.
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

# And finally write it to a StationXML file. We also force a validation against
# the StationXML schema to ensure it produces a valid StationXML file.
#
# Note that it is also possible to serialize to any of the other inventory
# output formats ObsPy supports.
inv.write("station.xml", format="stationxml", validate=True)